In [1]:
using Revise
using SourceSeparationNF
using LinearAlgebra, Random
using BenchmarkTools, ProfileView

BLAS.set_num_threads(1)

┌ Info: Precompiling SourceSeparationNF [0967cf83-991d-4407-9a5d-c519cfeee958]
└ @ Base loading.jl:1423


In [2]:
function create_neural_network(dim)

    return Model(dim,
        (
            DenseLayer(dim, 16),
            ReluLayer(16),
            DenseLayer(16, 16),
            ReluLayer(16),
            DenseLayer(16, dim)
        )
    )

end

create_neural_network (generic function with 1 method)

In [3]:
model = Model(
    4,
    (
        MvAdditiveLayer(4, create_neural_network(2)),
        PermutationLayer(4),
        MvAdditiveLayer(4, create_neural_network(2)),
        PermutationLayer(4),
        MvAdditiveLayer(4, create_neural_network(2)),
        PermutationLayer(4),
        MvAdditiveLayer(4, create_neural_network(2))
    )
);

In [4]:
@benchmark forward!($(model), $(randn(4)))

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.000 μs …  18.262 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.225 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.417 μs ± 650.088 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅▇██▇▅▄▃▂▂▁▁▁▁                         ▁                   ▂
  █████████████████▇█▇▇▆▆▄▅▆▆▄▅▅▅▄▅▆▅▆▆▇▇██▇███▇▇▇▇▇▇▇▇▇█▇▇▆▅ █
  3 μs         Histogram: log(frequency) by time         6 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
@benchmark propagate_error!($(model), $(randn(4)))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.770 μs …  5.720 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.820 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.826 μs ± 96.983 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▇ ▄        █                                            
  ▂▁▅▁█▁█▁▄▁▃▁▇▁▁█▁█▁▃▁▃▁▂▁▂▁▂▁▁▂▁▂▁▂▁▂▁▂▁▂▁▂▁▁▂▁▂▁▂▁▂▁▂▁▂▁▂ ▃
  1.77 μs        Histogram: frequency by time        2.04 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [6]:
@benchmark update!($model)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  5.443 μs …  18.257 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.671 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.686 μs ± 365.204 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▆▅▃▁▁ ▁▅▆▅▄▁   ▃█▅▃▃▁   ▁▂▁       ▂                        ▂
  ██████▄███████▇▆████████▇███▇▇▅▄▇███▇▆▅▅▃▄▁▃▄▅▁▁▃▃▃▁▁▄▁▁▃▁▃ █
  5.44 μs      Histogram: log(frequency) by time      6.43 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.